In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import sklearn.model_selection

from classes import *
from files import create_outfile_each_fold, create_outfile_mean, get_path_outfile_each_fold, get_path_mean, save_figs
from gpu import set_avx_avx2, set_gpu
from metrics import calculate_iou_dice, dice_loss, jaccard_distance_loss, jaccard_distance, dice_coef
from model import cfg_model, train_model

In [2]:
tensorflow.__version__

'2.8.2'

# GPU

In [3]:
set_avx_avx2()
set_gpu()

GPU: /physical_device:GPU:0


In [4]:
cfg = {
    "channel": 1,
    "batch_size": 4,
    "cross_validation": 5,
    "epochs": 75,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_base": "dataset",
    "path_out": "out"
}

In [5]:
path_images = os.path.join(cfg["path_base"], "grayscale/images_grayscale_400_400")
path_mask = os.path.join(cfg["path_base"], "mask")
piperaceae_dataset = Dataset(cfg, path_images, path_mask)
len(getattr(piperaceae_dataset, "list_images"))

375

In [6]:
list_images = getattr(piperaceae_dataset, "list_images")
x = numpy.array(list([l for l in list_images]), dtype=object)
y = numpy.array(list([getattr(l["image"], "label").value for l in list_images]))

collections.Counter(y.tolist())

Counter({'manekia': 75,
         'ottonia': 75,
         'peperomia': 75,
         'piper': 75,
         'pothomorphe': 75})

In [7]:
index = sklearn.model_selection.StratifiedKFold(n_splits=cfg["cross_validation"], shuffle=True, random_state=cfg["random_state"])
list_index = list([Index(fold, index_train, index_test) for fold, (index_train, index_test) in enumerate(index.split(x, y))])
len(list_index)

5

In [8]:
import itertools
list_index_test = list([getattr(i, "index_test").tolist() for i in list_index])
collections.Counter(sorted(list(itertools.chain(*list_index_test))))

Counter({0: 1,
         1: 1,
         2: 1,
         3: 1,
         4: 1,
         5: 1,
         6: 1,
         7: 1,
         8: 1,
         9: 1,
         10: 1,
         11: 1,
         12: 1,
         13: 1,
         14: 1,
         15: 1,
         16: 1,
         17: 1,
         18: 1,
         19: 1,
         20: 1,
         21: 1,
         22: 1,
         23: 1,
         24: 1,
         25: 1,
         26: 1,
         27: 1,
         28: 1,
         29: 1,
         30: 1,
         31: 1,
         32: 1,
         33: 1,
         34: 1,
         35: 1,
         36: 1,
         37: 1,
         38: 1,
         39: 1,
         40: 1,
         41: 1,
         42: 1,
         43: 1,
         44: 1,
         45: 1,
         46: 1,
         47: 1,
         48: 1,
         49: 1,
         50: 1,
         51: 1,
         52: 1,
         53: 1,
         54: 1,
         55: 1,
         56: 1,
         57: 1,
         58: 1,
         59: 1,
         60: 1,
         61: 1,
         62: 1,
  

In [9]:
def get_image(cfg, list_images):
    images = list([getattr(image["image"], "image") for image in list_images])
    mask = list([getattr(image["mask"], "image") for image in list_images])
    x = numpy.array(images).reshape((len(images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
    y = numpy.array(mask).reshape((len(mask), cfg["image_size"], cfg["image_size"], 1))
    return x, y


list_result = list([])
list_elapsed_time = list([])
path = get_path_mean(cfg)
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
for index in list_index:
    fold = str(getattr(index, "fold"))
    print(fold)
    path_outfile_each_fold = get_path_outfile_each_fold(fold, path)
    pathlib.Path(path_outfile_each_fold).mkdir(parents=True, exist_ok=True)

    index_train = getattr(index, "index_train")
    index_test = getattr(index, "index_test")

    x_train_o, y_train_o = x[index_train], y[index_train]
    x_test_o, y_test_o = x[index_test], y[index_test]
    x_train_o, x_val_o, y_train_o, y_val_o = sklearn.model_selection.train_test_split(x_train_o, y_train_o, test_size=cfg["val_size"], random_state=cfg["random_state"])

    x_train, y_train = get_image(cfg, x_train_o)
    x_test, y_test = get_image(cfg, x_test_o)
    x_val, y_val = get_image(cfg, x_val_o)

    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)
    print(x_val.shape, y_val.shape)

    checkpointer, reduce_learning_rate, steps_per_epoch, strategy, train_generator, unet_filename = cfg_model(cfg, fold, path_outfile_each_fold, x_train, y_train)

    if os.path.exists(unet_filename):
        model = tensorflow.keras.models.load_model(unet_filename, custom_objects = {"jaccard_distance_loss": jaccard_distance_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })
    else:
        elapsed_time, model = train_model(cfg, checkpointer, fold, path_outfile_each_fold, reduce_learning_rate, steps_per_epoch, strategy, train_generator, x_val, y_val)

    metrics = calculate_iou_dice(model, x_test, x_train, x_val, y_test, y_train, y_val)
    list_elapsed_time.append(elapsed_time)
    list_result.append(metrics)
    create_outfile_each_fold(elapsed_time, fold, metrics, path_outfile_each_fold)
    save_figs(cfg, model, path_outfile_each_fold, x_test_o, x_train_o, x_val_o, y_test_o, y_train_o, y_val_o)

0
(285, 400, 400, 1) (285, 400, 400, 1)
(75, 400, 400, 1) (75, 400, 400, 1)
(15, 400, 400, 1) (15, 400, 400, 1)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast 

C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


Epoch 1/75
22/72 [========>.....................] - ETA: 20s - loss: 0.7049 - dice_coef: 0.4504 - jaccard_distance: 0.2951 - precision: 0.7672 - recall: 0.4876


KeyboardInterrupt



In [ ]:
create_outfile_mean(cfg, list_elapsed_time, list_result, path)
len(list_result)